# Spam_ham_filter_project

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
df = pd.read_csv('spam_ham_dataset.csv.zip')
df.replace({r'\r\n':' '}, regex=True, inplace=True)
df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'spam_ham_dataset.csv.zip'

In [10]:
# prompt: Using dataframe df:

df.tail()

,Unnamed: 0,label,text,label_num
5166,1518,ham,Subject: put the 10 on the ft the transport vo...,0
5167,404,ham,Subject: 3 / 4 / 2000 and following noms hpl c...,0
5168,2933,ham,Subject: calpine daily gas nomination > > juli...,0
5169,1409,ham,Subject: industrial worksheets for august 2000...,0
5170,4807,spam,Subject: important online banking alert dear v...,1


In [11]:
from sklearn.feature_extraction.text import TfidfVectorizer

# Assuming 'df' is your DataFrame and 'text' is the column with text data
tfidf_vectorizer = TfidfVectorizer(max_features=5000)
X = tfidf_vectorizer.fit_transform(df['text'])

In [12]:
df.head()

,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291 thi...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001 ( see at...",0
2,3624,ham,"Subject: neon retreat ho ho ho , we ' re aroun...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs this deal is to b...,0


In [13]:
df.isna().sum()

Unnamed: 0    0
label         0
text          0
label_num     0
dtype: int64

In [15]:
df.shape

(5171, 4)

In [67]:
df.duplicated().sort_values(ascending=False)

0       False
3474    False
3452    False
3451    False
3450    False
        ...  
1722    False
1721    False
1720    False
1719    False
5170    False
Length: 5171, dtype: bool

In [68]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 5 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     5171 non-null   int64 
 1   label          5171 non-null   object
 2   text           5171 non-null   object
 3   label_num      5171 non-null   int64 
 4   label_encoded  5171 non-null   int64 
dtypes: int64(3), object(2)
memory usage: 202.1+ KB
None


In [72]:
# Check for non-numeric values in a column supposed to be numeric
print(df[pd.to_numeric(df['label'], errors='coerce').isna()].index)

RangeIndex(start=0, stop=5171, step=1)


In [75]:
# Check for non-numeric values in a column supposed to be numeric
print(df[pd.to_numeric(df['text'], errors='coerce').isna()].index)

RangeIndex(start=0, stop=5171, step=1)


In [76]:
!pip install nltk

In [77]:
%matplotlib inline
import nltk
import string
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier as RFR
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [78]:
ps = PorterStemmer()
corpus = []

all_stop_words = set(stopwords.words('english'))
all_stop_words.remove('not')

for i in range (len(df)):
    text = df['text'][i].lower().translate(str.maketrans('','', string.punctuation)).split()
    text = [ps.stem(word) for word in text if word not in all_stop_words]
    text = ' '.join(text)
    corpus.append(text)

## Uncomment the "corpus" if you want to see the result
# corpus

In [79]:
cv = CountVectorizer(max_features= 42500)
X = cv.fit_transform(corpus).toarray()
y = df['label_num']

In [30]:
len(X[0])

42500

In [80]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [81]:
def model_score(y_true,y_pred):
    acc_scor = accuracy_score(y_true, y_pred)
    prec_scor = precision_score(y_true, y_pred)
    recall_scor = recall_score(y_true, y_pred)
    f1_scor = f1_score(y_true, y_pred)
    overall_avg_score = (acc_scor + prec_scor + recall_scor + f1_scor) / 4

    print(f'Model accuracy score: {acc_scor}')
    print(f'Model precision score: {prec_scor}')
    print(f'Model recall score: {recall_scor}')
    print(f'Model f1 score: {f1_scor}')
    print(f'Average overall score performance: {overall_avg_score}')

    print(confusion_matrix(y_true, y_pred))

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Example of text preprocessing
# Assuming 'df' is your DataFrame and 'text' is the column with text data
tfidf_vectorizer = TfidfVectorizer(max_features=1000)
X = tfidf_vectorizer.fit_transform(df['text'])
y = df['label']  # Assuming you have a label for some ML task

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train, y_train)
predictions = model.predict(X_test)

# Evaluation
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

         ham       0.99      0.98      0.98       742
        spam       0.95      0.97      0.96       293

    accuracy                           0.98      1035
   macro avg       0.97      0.98      0.97      1035
weighted avg       0.98      0.98      0.98      1035



In [73]:
np.column_stack((y_test[:15], y_pred[:15]))

array([['ham', 0],
       ['spam', 0],
       ['ham', 0],
       ['ham', 1],
       ['ham', 0],
       ['ham', 0],
       ['ham', 0],
       ['ham', 0],
       ['ham', 0],
       ['ham', 1],
       ['ham', 1],
       ['ham', 0],
       ['spam', 0],
       ['spam', 0],
       ['spam', 0]], dtype=object)

In [82]:
type(model)

sklearn.ensemble._forest.RandomForestClassifier

In [83]:
df.dtypes

Unnamed: 0        int64
label            object
text             object
label_num         int64
label_encoded     int64
dtype: object

In [84]:
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
label_encoder = LabelEncoder()

# Assuming 'df' is your DataFrame and 'label' is your column with 'ham' or 'spam'
df['label_encoded'] = label_encoder.fit_transform(df['label'])

# Now df['label_encoded'] will have 0 for 'ham' and 1 for 'spam', or vice versa.

In [86]:
# Check for non-numeric values in a column supposed to be numeric
print(df[pd.to_numeric(df['label'], errors='coerce').isna()])

      Unnamed: 0 label                                               text  \
0            605   ham  Subject: enron methanol ; meter # : 988291 thi...   
1           2349   ham  Subject: hpl nom for january 9 , 2001 ( see at...   
2           3624   ham  Subject: neon retreat ho ho ho , we ' re aroun...   
3           4685  spam  Subject: photoshop , windows , office . cheap ...   
4           2030   ham  Subject: re : indian springs this deal is to b...   
...          ...   ...                                                ...   
5166        1518   ham  Subject: put the 10 on the ft the transport vo...   
5167         404   ham  Subject: 3 / 4 / 2000 and following noms hpl c...   
5168        2933   ham  Subject: calpine daily gas nomination > > juli...   
5169        1409   ham  Subject: industrial worksheets for august 2000...   
5170        4807  spam  Subject: important online banking alert dear v...   

      label_num  label_encoded  
0             0              0  
1        

In [87]:
df.dtypes

Unnamed: 0        int64
label            object
text             object
label_num         int64
label_encoded     int64
dtype: object

In [89]:
from sklearn.ensemble import RandomForestClassifier

In [91]:
# Create an instance of the RandomForestClassifier class
model = RandomForestClassifier()

# Fit the model on the training data
model.fit(X_train, y_train)

# Make predictions on the test data using the test features
y_pred = model.predict(X_test)

In [92]:
y_pred = model.predict(X_test)

In [93]:
print(y_test.shape)
print(y_pred.shape)

(1035,)
(1035,)


In [95]:
np.column_stack((y_test[:15], y_pred[:15]))

array([[0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0]])

In [96]:
model_score(y_test,y_pred)

Model accuracy score: 0.9748792270531401
Model precision score: 0.948220064724919
Model recall score: 0.966996699669967
Model f1 score: 0.9575163398692811
Average overall score performance: 0.9619030828293269
[[716  16]
 [ 10 293]]


In [ ]:
df.describe()

,Unnamed: 0,label_num
count,5171.000000,5171.000000
mean,2585.000000,0.289886
std,1492.883452,0.453753
min,0.000000,0.000000
25%,1292.500000,0.000000
50%,2585.000000,0.000000
75%,3877.500000,1.000000
max,5170.000000,1.000000


from matplotlib import pyplot as plt
_df_0['Unnamed: 0'].plot(kind='hist', bins=20, title='Unnamed: 0')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['label_num'].plot(kind='hist', bins=20, title='label_num')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='Unnamed: 0', y='label_num', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_3['Unnamed: 0'].plot(kind='line', figsize=(8, 4), title='Unnamed: 0')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_4['label_num'].plot(kind='line', figsize=(8, 4), title='label_num')
plt.gca().spines[['top', 'right']].set_visible(False)